In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - mask_essf.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Create a mask for the effective survey selection function grid which removes 
any fields where data is not considered for modelling.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy, dill as pickle, glob

## Matplotlib
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

# Set APOGEE version for the package
apogee_results_vers = 'l33'
# Year 7 is appropriate for DR16 (l33)
apo_combined_select_year = 7
os.environ['RESULTS_VERS'] = apogee_results_vers

import apogee.select as apsel
import apogee.tools as apotools

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import util as putil
from ges_mass import mass as pmass
from ges_mass import ssf as pssf

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','FEH_MIN','FEH_MAX','FEH_MIN_GSE',
            'FEH_MAX_GSE']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,feh_min,\
    feh_max,feh_min_gse,feh_max_gse = putil.parse_config_dict(cdict,keywords)

# Filenames
data_dir = base_dir+'data/'
version_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
ga_dir = data_dir+'gaia_apogee/'+version_dir
fig_dir = './fig/'
os.makedirs(fig_dir, exist_ok=True)
apogee_SF_filename = ga_dir+'apogee_SF.dat'
# Multiple possible effective selection function grids for different Fe/H ranges
apogee_effSF_filenames = glob.glob(ga_dir+'apogee_effSF_grid_inclArea*feh*.dat')

# Main eSSF mask output filename
mask_effSF_filename = ga_dir+'apogee_effSF_grid_mask.npy'
# Also save one which doesn't exclude globular cluster fields
mask_effSF_incl_gc_filename = ga_dir+'apogee_effSF_grid_mask_incl_gc.npy'
# Also save one which masks out the whole disk
mask_effSF_no_disk_filename = ga_dir+'apogee_effSF_grid_mask_no_disk.npy'

### Load data

In [ ]:
# Selection function
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
if os.path.exists(apogee_SF_filename):
    print('Loading APOGEE selection function from '+apogee_SF_filename)
    with open(apogee_SF_filename, 'rb') as f:
        apogee_SF = pickle.load(f)
else:
    sys.exit('Could not find APOGEE selection function, make it. Exiting...')

In [ ]:
# Check to see if all of the effective selection functions with different 
# [Fe/H] ranges will behave the same. They should
for i in range(len(apogee_effSF_filenames)):
    print('Examining Nspec mask for effective selection function: '+apogee_effSF_filenames[i])
    with open(apogee_effSF_filenames[i],'rb') as f:
        apof = pickle.load(f)
    _apof_nspec_mask = ~np.all(apof < 1e-9, axis=1)
    if i == 0:
        apof_nspec_mask = _apof_nspec_mask
    else:
        assert np.all(apof_nspec_mask==_apof_nspec_mask),\
            'Not all effective selection functions have the same Nspec mask. '+\
            'Should manually determine masks for each effective selection '+\
            'function grid'
print('\nAll selection functions have the same Nspec mask, loading the grid '
      'corresponding to FEH_MIN and FEH_MAX')

# Load the effective selection function grid
apogee_effSF_filename = ga_dir+'apogee_effSF_grid_inclArea_'+\
                        str(feh_min)+'_feh_'+str(feh_max)+'.dat'
if os.path.exists(apogee_effSF_filename):
    print('\nLoading effective selection function from '+apogee_effSF_filename)
    with open(apogee_effSF_filename,'rb') as f:
        apogee_effSF_grid_inclArea = pickle.load(f)
else:
    sys.exit('\nFailed to load effective survey section function')

### Field and eSSF grid information

In [ ]:
# Field info
apogee_field = apogee_SF._apogeeField
n_field = len(apogee_field)
field_glon = apogee_field['GLON']
field_glat = apogee_field['GLAT']
field_locid = apogee_field['LOCATION_ID']

# Distance modulus grid
dmod,dist = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Some sanity checks
assert len(field_glon) == apogee_effSF_grid_inclArea.shape[0]
assert np.all(field_locid == apogee_SF._locations)

### Now mask out fields where data is not considered

This includes:
1. Fields within a 20 degree square centered on the galactic bulge
2. Fields with a globular cluster in them
3. Fields with no spectroscopic targets

In [ ]:
# Cut bulge fields. Within 20 degrees of the galactic center
mask_bulge = ~(((field_glon > 340.) |\
                (field_glon < 20.)) &\
               (np.fabs(field_glat) < 20.)
              )

# Cut fields containing enhancements of globular cluster stars
gc_locid = pssf.get_globular_cluster_fields()
mask_gc = ~np.isin(field_locid,gc_locid)

# Mask parts of the selection function where there are no spectroscopic 
# targets and the whole field is set to zero.
mask_nspec = ~np.all(apogee_effSF_grid_inclArea < 1e-9, axis=1)

# Mask the disk at |b|<20
mask_disk = (np.fabs(field_glat) > 20.)

# The fitting mask, which represents the parts of the effective selection 
# function grid which are used in the fitting process. Make one with and 
# one without the GC mask
mask_effSF = mask_bulge & mask_gc & mask_nspec
mask_effSF_no_disk = mask_bulge & mask_gc & mask_nspec & mask_disk
mask_effSF_incl_gc = mask_bulge & mask_nspec

### Show the masks

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_bulge], plot_glat[~mask_bulge], 
           s=8, c='Red', zorder=1, label='Fields inside bulge mask')
ax.scatter(plot_glon[mask_bulge], plot_glat[mask_bulge], 
           c='ForestGreen', s=4, zorder=2, label='Fields outside bulge mask')

ax.plot([20,20],[20,-20],c='Black',linewidth=1.,zorder=3)
ax.plot([20,-20],[-20,-20],c='Black',linewidth=1.,zorder=3)
ax.plot([-20,-20],[-20,20],c='Black',linewidth=1.,zorder=3)
ax.plot([-20,20],[20,20],c='Black',linewidth=1.,zorder=3)

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()
ax.legend()

fig.savefig(fig_dir+'bulge_mask.png')
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_gc], plot_glat[~mask_gc], s=8, c='Red', 
           zorder=1, label='GC fields')
ax.scatter(plot_glon[mask_gc], plot_glat[mask_gc], 
           c='ForestGreen', s=4, zorder=2, 
           label='Fields without GC contamination')

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()
ax.legend()

fig.savefig(fig_dir+'gc_mask.png')
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_nspec], plot_glat[~mask_nspec], s=8, c='Red', 
           zorder=1, label='Fields with no spectroscopic targets')
ax.scatter(plot_glon[mask_nspec], plot_glat[mask_nspec], 
           c='ForestGreen', s=4, zorder=2, 
           label='Fields with spectroscopic targets')

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()
ax.legend()

fig.savefig(fig_dir+'nspec_mask.png')
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_disk], plot_glat[~mask_disk], s=8, c='Red', 
           zorder=1, label='Fields not in the fitting sample')
ax.scatter(plot_glon[mask_disk], plot_glat[mask_disk], 
           c='ForestGreen', s=4, zorder=2, 
           label='Fields in the fitting sample')

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()
ax.legend()

fig.savefig(fig_dir+'disk_mask.png')
fig.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)

plot_glon = copy.deepcopy(field_glon)
plot_glat = copy.deepcopy(field_glat)
plot_glon[plot_glon > 180] = plot_glon[plot_glon > 180] - 360

ax.scatter(plot_glon[~mask_effSF], plot_glat[~mask_effSF], s=8, c='Red', 
           zorder=1, label='Fields not in the fitting sample')
ax.scatter(plot_glon[mask_effSF], plot_glat[mask_effSF], 
           c='ForestGreen', s=4, zorder=2, 
           label='Fields in the fitting sample')

ax.set_xlabel(r'$\ell$ [deg]')
ax.set_ylabel(r'$b$ [deg]')
ax.set_xlim(-180,180)
ax.set_ylim(-90,90)
ax.invert_xaxis()
ax.legend()

fig.savefig(fig_dir+'essf_mask.png')
fig.show()

Note that many fields overlap with fields containing globular clusters yet they 
may not contain significant numbers of globular clusters themselves.

### Save the masks

In [ ]:
np.save(mask_effSF_filename, mask_effSF)
np.save(mask_effSF_no_disk_filename, mask_effSF_no_disk)
np.save(mask_effSF_incl_gc_filename, mask_effSF_incl_gc)